This one is getting ./AS13_reduc/data/AS13_stgdir_labelGeneric.csv

In [ ]:
pip install setfit

In [ ]:
##### CONFIG ####

dataset_name = "AS13_stgdir_labelGeneric"

# note: in AS' initial tests, "s3" was used, but "s1" and "s2" have many more downloads
# "s1" has better results than "s2" in the benchmark and only 13 lgs, including FR: https://www.sbert.net/docs/pretrained_models.html
# "s2" has 50 lgs (see benchmark)

sbert_prefix = "sentence-transformers/"
sbert_model_dict = {"s1": "distiluse-base-multilingual-cased-v1", "s2": "distiluse-base-multilingual-cased-v2",
                    "s3": "distiluse-base-multilingual-cased"}

model_key = "s1"
model_ckpt = sbert_prefix + sbert_model_dict[model_key]
fraction_kept = 5 # can only be 100, 50, 25, 10, 5
run_number = 1

batch_size = 16
max_iterations_contr_learn = 20 # number of iterations for contrastive learning
max_epochs = 6

use_drive = False
save_model = False

dirname_for_drive = f"{model_ckpt}__{dataset_name}__{fraction_kept}__{run_number}"

# output dir format (below)
#output_dir=f"{model_ckpt}__finetuned__{dataset_name}__{fraction_kept}__{run_number}"

In [ ]:
assert fraction_kept in [100*x for x in (1, 0.5, 0.25, 0.1, 0.05)]
assert run_number in (1,2,3)

In [ ]:
# fraction kept to color map
clrmap_dict = {100: "Greens", 50: "Blues", 25: "Purples", 10: "Oranges", 5: "Reds"}
assert fraction_kept in clrmap_dict

In [ ]:
!python --version

In [ ]:
!mkdir data

In [ ]:
! pip install transformers

In [ ]:
! pip install datasets

In [ ]:
! pip install evaluate

In [ ]:
!pip install sentencepiece

In [ ]:
import os

from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
import evaluate
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
import torch

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

J'ai téléversé les données sur mon serveur

In [ ]:
![[ ! -d data ]] && mkdir data || echo "data directory exists"
#![[ ! -f data/AS13_stgdir_clean.csv ]] && wget -P data https://prf1.org/stgs/as-did/AS13_stgdir_clean.csv || echo "dataset already downloaded"
![[ ! -f data/AS13_stgdir_labelGeneric.csv ]] && wget -P data https://prf1.org/stgs/as-did/AS13_stgdir_labelGeneric.csv || echo "dataset already downloaded"

Recette pour éléchargement fichiers qui sont sur Drive sur la session Colab (sans monter le Drive) (pas nécessaire ici mais comme idée pour d'autres fichiers).
C'est un fichier quelconque non relié à la tâche qui a été utilisé pour tester.

In [ ]:
mydf = "data/AS13_stgdir_labelGeneric.csv"

In [ ]:
labelCol = 'labelGeneric'

In [ ]:
import pandas as pd
AS13_df = pd.read_csv(mydf, sep="|", dtype={'description': 'object',
                                            labelCol: 'category'})

In [ ]:
AS13_df = AS13_df.drop_duplicates()
AS13_df = AS13_df.dropna(how='any')
AS13_df.info()

In [ ]:
df = AS13_df

In [ ]:
df['labelGeneric'].value_counts()

In [ ]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(AS13_df[labelCol].unique().categories.to_list())
label2id = {class_names[i]:i for i in range(len(class_names))}
id2label = {i:class_names[i] for i in range(len(class_names))}

In [ ]:
#class_names, label2id, id2label

In [ ]:
# nv dataFrame
data_df = pd.DataFrame()
data_df['text'] = AS13_df.description
# La classe cible est la région (province) sous forme d'identifiant numérique
data_df['label'] = AS13_df[labelCol].map(label2id)

In [ ]:
data_df = data_df.reset_index(drop=True)

In [ ]:
# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
stgtype_features = Features({'text': Value('string'),
                             'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=stgtype_features)

In [ ]:
# Découpage en (train + validation) (this part will be reduced to 50%, 25% and so on to compare quality) et (test).
# The test part is supposed to always stay the same size and with the same examples

In [ ]:
TRAIN_FULL, TRAIN_REDUCED = False, False
size_to_exclude = (100 - fraction_kept)/100
if size_to_exclude == 0:
  TRAIN_FULL = True
  print("FULL TRAIN")
else:
  TRAIN_REDUCED = True
  print(f"REDUCED TRAIN, KEEPING {fraction_kept}%")

In [ ]:
print(f"SIZE TO EXCLUDE IS {size_to_exclude}")

In [ ]:
if TRAIN_REDUCED:
  trainvalid_test = data.train_test_split(test_size=0.2, shuffle=True, seed=12)
  trainvalid_reduced = trainvalid_test['train'].train_test_split(test_size=size_to_exclude, shuffle=True, seed=12)
  trainvalid_reduced_trainvalid = trainvalid_reduced['train'].train_test_split(test_size=0.2, shuffle=True, seed=12)
  dataset = DatasetDict({"train": trainvalid_reduced_trainvalid["train"],
                         "valid": trainvalid_reduced_trainvalid["test"],
                         "test": trainvalid_test["test"]})
elif TRAIN_FULL:
  trainvalid_test = data.train_test_split(test_size=0.2, shuffle=True, seed=12)
  trainvalid_trainvalid = trainvalid_test["train"].train_test_split(test_size=0.2, shuffle=True, seed=12)
  dataset = DatasetDict({"train": trainvalid_trainvalid["train"],
                         "valid": trainvalid_trainvalid["test"],
                         "test": trainvalid_test["test"]})

In [ ]:
data = dataset

In [ ]:
data

In [ ]:
# this is unused unless use steps as strategy in training args
epoch_size = len(data['train'])/batch_size

In [ ]:
print(f"EPOCH SIZE IS: {epoch_size}")

When needed, export the dataset to verify later that it's indeed the same for all experiments

In [ ]:
from pathlib import Path

if use_drive:
  from google.colab import drive
  drive.mount("/content/drive")
  dataset_bkps = "/content/drive/MyDrive/___wv/datasets"
  os.mkdir(os.path.join(dataset_bkps, dirname_for_drive))
  for split, split_data in data.items():
    out_path = Path(dataset_bkps) / dirname_for_drive / f"{dirname_for_drive}-{split}.csv"
    print(out_path.absolute())
    split_data.to_csv(Path(dataset_bkps) / dirname_for_drive / f"{dirname_for_drive}-{split}.csv")

In [ ]:
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-train.csv
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-valid.csv
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-test.csv

In [ ]:
# Taille du vocabulaire
#tokenizer.vocab_size

In [ ]:
# Taille de contexte maximum
#tokenizer.model_max_length

In [ ]:
from math import ceil
intended_eval_steps = ceil(len(data['train'])/batch_size)

In [ ]:
batch_size

In [ ]:
print(f"\n### Train with Model:[{model_ckpt}] || IterContrLrn:[{max_iterations_contr_learn}] || BatchSize:[{batch_size}] || Epochs:[{max_epochs}] ###\n")
print(f"\n### Keeping {fraction_kept}% of data. Excludes {size_to_exclude*100}% ###\n")

In [ ]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained(model_ckpt)

In [ ]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=data['valid'],
    loss_class=CosineSimilarityLoss,
    batch_size=batch_size,
    num_iterations=max_iterations_contr_learn, # Number of text pairs to generate for contrastive learning
    num_epochs=max_epochs # Number of epochs to use for contrastive learning
)

In [ ]:
from datetime import datetime
start_time = datetime.now()

In [ ]:
trainer.train()
metrics = trainer.evaluate()

In [ ]:
end_time = datetime.now()
time_diff = end_time - start_time
print(f"Training time {time_diff.total_seconds()} seconds")

In [ ]:
metrics

In [ ]:
# Test-set formatting and tests

test_data_as_text = data["test"].to_list()

In [ ]:
test_data_as_text[0]

In [ ]:
test_examples = [ve['text'] for ve in test_data_as_text]

In [ ]:
test_labels = [ve['label'] for ve in test_data_as_text]

In [ ]:
test_examples[0]

In [ ]:
testset_preds = model(test_examples)

In [ ]:
label_names = data['test'].features['label'].names

In [ ]:
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=labels_for_fig)
    #disp.plot(cmap="Purples", values_format=".2f", ax=ax, colorbar=False)
    disp.plot(cmap=clrmap_dict[fraction_kept], values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

plot_confusion_matrix(testset_preds.tolist(), test_labels, label_names)

In [ ]:
print("Classification report:\n\n{}".format(classification_report(test_labels, testset_preds.tolist(), digits=4)))

In [ ]:
for i in range(13):print(i, id2label[i], end=" ")

In [ ]:
print(f"\n### Train with Model:[{model_ckpt}] || IterContrLrn:[{max_iterations_contr_learn}] || BatchSize:[{batch_size}] || Epochs:[{max_epochs}]###\n")
print(f"\n### Keeping {fraction_kept}% of data. Excludes {size_to_exclude*100}% ###\n")

In [ ]:
if save_model:
  assert use_drive is True
  model.save_pretrained(f"/content/drive/MyDrive/___wv/{model_ckpt}-finetuned-stgdir-AS13reduc_50_common-testset")
  tokenizer.save_pretrained(f"/content/drive/MyDrive/___wv/{model_ckpt}-finetuned-stgdir-AS13reduc_50_common-testset")